In [1]:
'''
Backtrader data class
'''
import backtrader.feeds as btfeed


import datetime

from backtrader import TimeFrame
from backtrader import date2num

from backtrader import Strategy

class StockHLOC(btfeed.GenericCSVData):
    params = (
        ('fromdate', datetime.datetime(1900, 12, 31)),
        ('todate', datetime.datetime(2100, 12, 31)),
        ('nullvalue', 0.0),
        ('dtformat', ('%Y-%m-%d %H:%M:%S')),
        ('tmformat', ('%H:%M:%S')),
        ('datetime', 0),
        ('time', -1),
        ('high', 2),
        ('low', 3),
        ('open', 4),
        ('close', 5),
        ('volume', 6),
        ('openinterest', -1)
    )

    def _loadline(self, linetokens):
        # Datetime needs special treatment
        dtfield = linetokens[self.p.datetime]
        if self._dtstr:
            dtformat = self.p.dtformat

            if self.p.time >= 0:
                # add time value and format if it's in a separate field
                dtfield += 'T' + linetokens[self.p.time]
                dtformat += 'T' + self.p.tmformat


            dt = datetime.datetime.strptime(dtfield, dtformat)
        else:
            dt = self._dtconvert(dtfield)

        if self.p.timeframe >= TimeFrame.Days:
            # check if the expected end of session is larger than parsed
            if self._tzinput:
                dtin = self._tzinput.localize(dt)  # pytz compatible-ized
            else:
                dtin = dt

            dtnum = date2num(dtin)  # utc'ize

            dteos = datetime.datetime.combine(dt.date(), self.p.sessionend)
            dteosnum = self.date2num(dteos)  # utc'ize

            if dteosnum > dtnum:
                self.lines.datetime[0] = dteosnum
            else:
                # Avoid reconversion if already converted dtin == dt
                self.l.datetime[0] = date2num(dt) if self._tzinput else dtnum
        else:
            self.lines.datetime[0] = date2num(dt)

        # The rest of the fields can be done with the same procedure
        for linefield in (x for x in self.getlinealiases() if x != 'datetime'):
            # Get the index created from the passed params
            csvidx = getattr(self.params, linefield)

            if csvidx is None or csvidx < 0:
                # the field will not be present, assignt the "nullvalue"
                csvfield = self.p.nullvalue
            else:
                # get it from the token
                csvfield = linetokens[csvidx]

            if csvfield == '':
                # if empty ... assign the "nullvalue"
                csvfield = self.p.nullvalue

            # get the corresponding line reference and set the value
            line = getattr(self.lines, linefield)
            line[0] = float(float(csvfield))

        return True


In [4]:
%matplotlib widget
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import backtrader as bt
import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])


module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


class MyStrategy(bt.Strategy):

    params = (
        ("macd_fast", 13),
        ("macd_slow", 34),
        ("macd_signal", 9),
        ("atr_period", 14),
        ("atr_stop", 13),
        ("stop_gain_multiple", 2),
        ("consecutive_peaks", 3),
    )
    
    def __init__(self):
        # self.dataclose = self.datas[0].close
        # self.open = self.datas[0].open

        self.macd = bt.indicators.MACD(self.data,
                                       period_me1=self.params.macd_fast,
                                       period_me2=self.params.macd_slow,
                                       period_signal=self.params.macd_signal)

        # plot MACD histogram
        bt.indicators.MACDHisto(self.data,
                                       period_me1=self.params.macd_fast,
                                       period_me2=self.params.macd_slow,
                                       period_signal=self.params.macd_signal)
        
        self.atr = bt.indicators.ATR(self.data, period=self.params.atr_period)

        self.macd_positive = True
        self.macd_peaks = []
        self.stock_values_in_macd_peak = []
        self.consecutive_peak_count = 0
        self.count = 27649

    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime
        print('%s, %s, %s' % (dt.date(0).isoformat(), dt.time(), txt))

    def next(self):
        # Simply log the closing price of the series from the reference
        # self.log('Close, %.2f' % self.macd[0])
        

        # count, and hard code a stop point
        # self.count = self.count - 1
        # if count = 0:
        #     return

        macd_positive = self.macd.macd[0] > 0
        
        # MACD pos/meg reverse, reset 
        if self.macd_positive != macd_positive:
            self.macd_peaks = []
            self.stock_values_in_macd_peak = []
            self.consecutive_peak_count = 0
            self.macd_positive = macd_positive
            return



        # Detect a local peak:
        if abs(self.macd.macd[-2]) < abs(self.macd.macd[-1]) and abs(self.macd.macd[-1]) > abs(self.macd.macd[0]):
            
            macd_peak = self.macd.macd[-1]
            stock_val = self.data.close[0]

            if len(self.macd_peaks) >= 3:
                self.macd_peaks = self.macd_peaks[1:]
                self.stock_values_in_macd_peak = self.stock_values_in_macd_peak[1:]
            # If there were peaks before the current peak, verify the previous peak was stronger than the current one
            if len(self.macd_peaks) > 1:
                # if the last peak is smaller than the current peak
                while abs(self.macd_peaks[-1]) < abs(macd_peak):
                    self.macd_peaks = self.macd_peaks[0:-1]
                    self.stock_values_in_macd_peak = self.stock_values_in_macd_peak[0:-1]
                    self.consecutive_peak_count = self.consecutive_peak_count - 1
                    
                self.consecutive_peak_count += 1
                self.macd_peaks.append(macd_peak)
                self.stock_values_in_macd_peak.append(stock_val)

            # Record the current peak
            if len(self.macd_peaks) == 0:
                self.consecutive_peak_count += 1
                self.macd_peaks.append(macd_peak)
                self.stock_values_in_macd_peak.append(stock_val)

        # verify signal

        
        print('peak: ', self.stock_values_in_macd_peak)
        if len(self.macd_peaks) == 3 and self.macd_positive:
            if self.stock_values_in_macd_peak[0] < self.stock_values_in_macd_peak[1] \
            and self.stock_values_in_macd_peak[1] < self.stock_values_in_macd_peak[2]:
                print('macd1: ', self.macd.macd[-2], 'macd2: ', self.macd.macd[-1], 'macd3: ', self.macd.macd[0])
                print('stock1: ', self.data.close[-2], 'stock2: ', self.data.close[-1], 'stock3: ', data.close[0])
                print("short !!!")


        if len(self.macd_peaks) == 3 and not self.macd_positive:
            if self.stock_values_in_macd_peak[0] > self.stock_values_in_macd_peak[1] \
            and self.stock_values_in_macd_peak[1] > self.stock_values_in_macd_peak[2]:
                print('len macd1: ', self.macd.macd[-2], 'len macd2: ', self.macd.macd[-1], 'len macd3: ', self.macd.macd[0])
                print('stock1: ', self.data.close[-2], 'stock2: ', self.data.close[-1], 'stock3: ', data.close[0])
                print("long !!!")

if __name__ == '__main__':
    modpath = os.path.dirname(os.path.abspath('../data/BTC_30mins_2y_parsed.CSV'))
    datapath = os.path.join(modpath, 'BTC_30mins_2y_parsed.CSV')

    data = StockHLOC(
        dataname=datapath,
        name="SPY_30mins_2y_parsed",
        datetime=0,
        high=2,
        low=3,
        open=4,
        close=5,
        volume=6,
        timeframe=bt.TimeFrame.Minutes,
        compression=60
    )
    cerebro = bt.Cerebro()
    cerebro.adddata(data)
    cerebro.addstrategy(MyStrategy)
    cerebro.broker.setcash(300_000.0)
    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    %matplotlib widget
    cerebro.plot(iplot=False)


Starting Portfolio Value: 300000.00
peak:  [46124.5]
peak:  [46124.5]
peak:  [46124.5]
peak:  [46124.5]
peak:  [46124.5]
peak:  [46124.5]
peak:  [46124.5]
peak:  [46124.5]
peak:  [46124.5]
peak:  [46124.5]
peak:  [46124.5]
peak:  [46124.5]
peak:  [46124.5]
peak:  [46124.5]
peak:  [46124.5]
peak:  [46124.5]
peak:  [46124.5]
peak:  [46124.5]
peak:  [46124.5]
peak:  [46124.5]
peak:  [46124.5]
peak:  [46124.5]
peak:  [46124.5]
peak:  [46124.5]
peak:  [46124.5]
peak:  [46124.5]
peak:  [46124.5]
peak:  [46124.5]
peak:  [46124.5]
peak:  [46124.5]
peak:  [46124.5]
peak:  [46124.5]
peak:  [46124.5]
peak:  [46124.5]
peak:  [46124.5]
peak:  []
peak:  []
peak:  []
peak:  []
peak:  []
peak:  []
peak:  []
peak:  []
peak:  []
peak:  []
peak:  []
peak:  []
peak:  []
peak:  []
peak:  []
peak:  [46731.75]
peak:  [46731.75]
peak:  [46731.75]
peak:  [46731.75]
peak:  [46731.75]
peak:  [46731.75]
peak:  [46731.75]
peak:  [46731.75]
peak:  [46731.75]
peak:  [46731.75]
peak:  [46731.75]
peak:  [46731.75]
pea

FigureCanvas object 0x110f65a90 wrapping NSView 0x7fc78f59d8e0

In [ ]:
%matplotlib widget
cerebro.plot(iplot=False)